## Garbage Classifier 🗑️ 🚮 🚯

### ⚙️ Setup

In [21]:
# Import libraries
import os
from dotenv import load_dotenv

In [22]:
# Reload for caching
from importlib import reload
import src.base_dataset as dataset
reload(dataset)
import src.dataset_loader as dataset_loader
reload(dataset_loader)
import src.garbage_model as garbage_model
reload(garbage_model)
import src.my_transforms as my_transforms
reload(my_transforms)

from src.base_dataset import BaseDataset
from src.garbage_model import GarbageModel
from src.dataset_loader import list_data_and_prepare_labels, split_data
from src.my_transforms import torch_vision_transform, torch_vision_transform_test

In [23]:
# get file directory
curr_dir = os.getcwd()

# load dataset_path
dotenv_path = os.path.join(curr_dir, ".env")
load_dotenv(dotenv_path)

True

In [24]:
# Constants
DEFAULT_EPOCHS = 12
TEST_SPLIT = 0.2
VAL_SPLIT = 0.2
BATCH_SIZE = 64

In [25]:
# Variables
dataset_path = os.getenv("DATASET_LOCAL_PATH")
normalized_path = dataset_path
normalized_path

'D:/chris/Documents/UofC/MEng Soft/winter/ENEL 645/ENEL 645/ENEL 645/A2/small_dataset'

### ↻ Data Loader

In [26]:
# get dataset
images_path = normalized_path + "/**/*.png"
images, labels_int, classes = list_data_and_prepare_labels(images_path)

In [27]:
# split dataset
all_dataset = split_data(images, labels_int, VAL_SPLIT, TEST_SPLIT)
train_set = all_dataset["Train"]
val_set = all_dataset["Validation"]
test_set = all_dataset["Validation"]

In [28]:
# Get the data loader for the train set
train_loader = BaseDataset(train_set, transform=torch_vision_transform)
val_loader = BaseDataset(val_set, transform=torch_vision_transform)
test_loader = BaseDataset(test_set,transform= torch_vision_transform_test)

In [29]:
""" Delete when cleaning up"""
print(train_loader.file_paths[0])
print(train_loader.labels[0])
type(train_loader.transform)

D:/chris/Documents/UofC/MEng Soft/winter/ENEL 645/ENEL 645/ENEL 645/A2/small_dataset\Train\Blue\ad_paper_19.png
1


torchvision.transforms.transforms.Compose

### 🏃‍♂️ Train

In [30]:
# load model
# net = GarbageModel()